# Libraries

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
from datetime import timedelta
import datetime 
import matplotlib.pyplot as plt
import io
import urllib, base64
from flask import Flask, request

# CSS

In [2]:
css_style="""<style>
body {
  display: flex;
  justify-content: center;
  align-items: center;
  height: 100vh;
}
form {
  background-color: #f2f2f2;
  padding: 20px;
  border-radius: 10px;
  box-shadow: 2px 2px 5px #888888;
  text-align: center;
}
input[type="email"], input[type="password"], input[type="date"] {
  padding: 10px;
  font-size: 16px;
  margin: 10px 0;
  width: 100%;
  box-sizing: border-box;
  border-radius: 5px;
  border: 1px solid #888888;
}
input[type="submit"] {
  padding: 10px 20px;
  background-color: #4CAF50;
  color: white;
  border-radius: 5px;
  border: none;
  font-size: 16px;
  cursor: pointer;
}
.data {
  width: 50%;
  margin: 20px auto;
  border-collapse: collapse;
}
.data td, .data th {
  border: 1px solid #dddddd;
  padding: 8px;
  text-align: left;
}
.data tr:nth-child(even) {
  background-color: #dddddd;
}
</style>"""

# Functions

In [4]:
app = Flask(__name__)

def myfitnesspal_signin (myfitnesspal_email, myfitnesspal_password):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://www.myfitnesspal.com/account/login")
    time.sleep(1)
    user_email = driver.find_element(By.ID, "email")
    user_email.send_keys(myfitnesspal_email)
    user_password = driver.find_element(By.ID, "password")
    user_password.send_keys(myfitnesspal_password)
    time.sleep(1)
    user_password.send_keys(Keys.RETURN)
    print("Enter!")    
    
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "css-m5vuyl")))

    #__next > div > main > div.css-1bbmu2g > div > div > div.MuiBox-root.css-8atqhb > div.MuiGrid-root.MuiGrid-container.css-ia8go6 > div.MuiGrid-root.MuiGrid-item.css-koo75d > div > div.MuiGrid-root.MuiGrid-container.css-1osfuow > div:nth-child(1) > p
    return driver

def get_totals (driver, date):  
    driver.get("https://www.myfitnesspal.com/food/diary/?date="+date)
    time.sleep(1)
    td_elements = driver.find_elements(by=By.CSS_SELECTOR, value="tr.total td")
    values = []
    for td_element in td_elements:
        values.append(td_element.text)
    values = [value for value in values if value != ""]
    chunks = [values[i:i + 7] for i in range(0, len(values), 7)]
    df = pd.DataFrame(chunks, columns=["date", "calories", "carbs", "fat", "protein", "sodium", "sugar"])
    df_totals = pd.DataFrame(df.iloc[0]).T
    df_totals.iloc[0]['date'] = date
    return df_totals

def get_myfitnesspal_data(myfitnesspal_email,myfitnesspal_password,start_date,end_date):
    driver = myfitnesspal_signin (myfitnesspal_email, myfitnesspal_password)
     
    df = pd.DataFrame()
    date_format = "%Y-%m-%d"
    start_date = datetime.datetime.strptime(start_date, date_format).date()
    end_date = datetime.datetime.strptime(end_date, date_format).date()
    
    for i in range((end_date - start_date).days + 1):
        current_date = start_date + timedelta(i)
        df = pd.concat ([df,get_totals (driver, str(current_date))])
    driver.quit()
    return df.to_html(index=False, classes='data')

@app.route("/", methods=["GET", "POST"])
def form_with_graphs():
    email = password = start_date = end_date = None
    if request.method == "POST":
        email = request.form.get("email")
        password = request.form.get("password")
        start_date = request.form.get("start_date")
        end_date = request.form.get("end_date")
        
        return """
            <html>
            <head>
              <title>Form Result</title>
              {}
            </head>
            <body>
            <form action="/" method="get">
                <h1>Remco's MyFitnessPal Client Control</h1>
                <label>Email: {}</label>
                <br>
                <label>From {} to {}</label>
                <br>
                {}
                <input type="submit" value="Go back to the form">
              </form>
            </body>
            </html>
        """.format(css_style, email, start_date, end_date, get_myfitnesspal_data(email,password,start_date,end_date))

    return """
        <html>
        <head>
          <title>Form Input</title>
          {}
        </head>
        <body>
          <form action="/" method="post">
          <h1>Remco's MyFitnessPal Client Control</h1>
            <label for="email">Enter Your Email:</label>
            <input type="email" id="email" name="email" required>
            <br>

            <label for="password">Enter Your Password:</label>
            <input type="password" id="password" name="password" required>
            <br>

            <label for="start_date">Enter Start Date:</label>
            <input type="date" id="start_date" name="start_date" required>
            <br>

            <label for="end_date">Enter End Date:</label>
            <input type="date" id="end_date" name="end_date" required>
            <br>
  
            <input type="submit" value="Submit">
          </form>
        </body>
        </html>
    """.format(css_style)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Feb/2023 22:24:04] "GET / HTTP/1.1" 200 -


====== WebDriver manager ======
Current google-chrome version is 110.0.5481
Get LATEST driver version for 110.0.5481
Driver [/Users/nw/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver] found in cache
<ipython-input-4-480c6fec4b3f>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Enter!


127.0.0.1 - - [11/Feb/2023 22:24:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2023 22:25:31] "GET / HTTP/1.1" 200 -
